# Getting Started: Exploring Nemo Fundamentals

NeMo is a toolkit for creating [Conversational AI](https://developer.nvidia.com/conversational-ai#started) applications.

NeMo toolkit makes it possible for researchers to easily compose complex neural network architectures for conversational AI using reusable components - Neural Modules. Neural Modules are conceptual blocks of neural networks that take typed inputs and produce typed outputs. Such modules typically represent data layers, encoders, decoders, language models, loss functions, or methods of combining activations.

The toolkit comes with extendable collections of pre-built modules and ready-to-use models for automatic speech recognition (ASR), natural language processing (NLP) and text synthesis (TTS). Built for speed, NeMo can utilize NVIDIA's Tensor Cores and scale out training to multiple GPUs and multiple nodes.

For more information, please visit https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/main/#

In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell.

## Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install text-unidecode

# ## Install NeMo
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

## Install TorchAudio
!pip install torchaudio>=0.10.0 -f https://download.pytorch.org/whl/torch_stable.html

## Grab the config we'll use in this example
!mkdir configs

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=e42ad079571d8a73167c3eb0d7aab866551ce3f70f0bbe9522385aaaa9ad7885
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsndfile1 is already the newest version (1.0.31-2ubuntu0.2).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base libsox3 libwavpack1
Suggested packages:
  libsox-fmt-all
The following NEW packages will be installed:
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base libsox3 libwavpack1 sox
0 upgraded, 7 newly installed, 0 to remove and 29 not upgraded.
Need to get 617 kB of archives.
After this operation, 1,764 kB of additi

## Foundations of NeMo
---------

NeMo models leverage [PyTorch Lightning](https://github.com/PyTorchLightning/pytorch-lightning) Module, and are compatible with the entire PyTorch ecosystem. This means that users have the full flexibility of using the higher level APIs provided by PyTorch Lightning (via Trainer), or write their own training and evaluation loops in PyTorch directly (by simply calling the model and the individual components of the model).

For NeMo developers, a "Model" is the neural network(s) as well as all the infrastructure supporting those network(s), wrapped into a singular, cohesive unit. As such, all NeMo models are constructed to contain the following out of the box (at the bare minimum, some models support additional functionality too!) -

 -  Neural Network architecture - all of the modules that are required for the model.

 -  Dataset + Data Loaders - all of the components that prepare the data for consumption during training or evaluation.

 -  Preprocessing + Postprocessing - all of the components that process the datasets so they can easily be consumed by the modules.

 -  Optimizer + Schedulers - basic defaults that work out of the box, and allow further experimentation with ease.

 - Any other supporting infrastructure - tokenizers, language model configuration, data augmentation etc.




In [ ]:
import nemo
nemo.__version__

## NeMo Collections

NeMo is sub-divided into a few fundamental collections based on their domains - `asr`, `nlp`, `tts`. When you performed the `import nemo` statement above, none of the above collections were imported. This is because you might not need all of the collections at once, so NeMo allows partial imports of just one or more collection, as and when you require them.

-------
Let's import the above three collections -

In [ ]:
import nemo.collections.asr as nemo_asr
import nemo.collections.nlp as nemo_nlp
import nemo.collections.tts as nemo_tts

## NeMo Models in Collections

NeMo contains several models for each of its collections, pertaining to certain common tasks involved in conversational AI. At a brief glance, let's look at all the Models that NeMo offers for the above 3 collections.

In [ ]:
asr_models = [model for model in dir(nemo_asr.models) if model.endswith("Model")]
asr_models

In [ ]:
nlp_models = [model for model in dir(nemo_nlp.models) if model.endswith("Model")]
nlp_models

In [ ]:
tts_models = [model for model in dir(nemo_tts.models) if model.endswith("Model")]
tts_models

## The NeMo Model

Let's dive deeper into what a NeMo model really is. There are many ways we can create these models - we can use the constructor and pass in a config, we can instantiate the model from a pre-trained checkpoint, or simply pass a pre-trained model name and instantiate a model directly from the cloud !

---------
For now, let's try to work with an ASR model - [Citrinet](https://arxiv.org/abs/2104.01721)

In [ ]:
citrinet = nemo_asr.models.EncDecCTCModelBPE.from_pretrained('stt_en_citrinet_512')

In [ ]:
citrinet.summarize()

## Model Configuration using OmegaConf
--------

So we could download, instantiate and analyse the high level structure of the `Citrinet` model in a few lines! Now let's delve deeper into the configuration file that makes the model work.

First, we import [OmegaConf](https://omegaconf.readthedocs.io/en/latest/). OmegaConf is an excellent library that is used throughout NeMo in order to enable us to perform yaml configuration management more easily. Additionally, it plays well with another library, [Hydra](https://hydra.cc/docs/intro/), that is used by NeMo to perform on the fly config edits from the command line, dramatically boosting ease of use of our config files !

In [ ]:
from omegaconf import OmegaConf

All NeMo models come packaged with their model configuration inside the `cfg` attribute. While technically it is meant to be config declaration of the model as it has been currently constructed, `cfg` is an essential tool to modify the behaviour of the Model after it has been constructed. It can be safely used to make it easier to perform many essential tasks inside Models.

To be doubly sure, we generally work on a copy of the config until we are ready to edit it inside the model

In [ ]:
import copy

In [ ]:
cfg = copy.deepcopy(citrinet.cfg)
print(OmegaConf.to_yaml(cfg))

## Analysing the contents of the Model config
----------

Above we see a configuration for the Citrinet model. As discussed in the beginning, NeMo models contain the entire definition of the neural network(s) as well as most of the surrounding infrastructure to support that model within themselves. Here, we see a perfect example of this behaviour.

Citrinet contains within its config -

- `preprocessor` - MelSpectrogram preprocessing layer
- `encoder` - The acoustic encoder model.
- `decoder` - The CTC decoder layer.
- `optim` (and potentially `sched`) - Optimizer configuration. Can optionally include Scheduler information.
- `spec_augment` - Spectrogram Augmentation support.
- `train_ds`, `validation_ds` and `test_ds` - Dataset and data loader construction information.

## Modifying the contents of the Model config
----------

Say we want to experiment with a different preprocessor (we want MelSpectrogram, but with different configuration than was provided in the original configuration). Or say we want to add a scheduler to this model during training.

OmegaConf makes this a very simple task for us!

In [ ]:
# OmegaConf won't allow you to add new config items, so we temporarily disable this safeguard.
OmegaConf.set_struct(cfg, False)

# Let's see the old optim config
print("Old Config: ")
print(OmegaConf.to_yaml(cfg.optim))

sched = {'name': 'CosineAnnealing', 'warmup_steps': 1000, 'min_lr': 1e-6}
sched = OmegaConf.create(sched)  # Convert it into a DictConfig

# Assign it to cfg.optim.sched namespace
cfg.optim.sched = sched

# Let's see the new optim config
print("New Config: ")
print(OmegaConf.to_yaml(cfg.optim))

# Here, we restore the safeguards so no more additions can be made to the config
OmegaConf.set_struct(cfg, True)

## Updating the model from config
----------

NeMo Models can be updated in a few ways, but we follow similar patterns within each collection so as to maintain consistency.

Here, we will show the two most common ways to modify core components of the model - using the `from_config_dict` method, and updating a few special parts of the model.

Remember, all NeMo models are PyTorch Lightning modules, which themselves are PyTorch modules, so we have a lot of flexibility here!

### Update model using `from_config_dict`

In certain config files, you will notice the following pattern :

```yaml
preprocessor:
  _target_: nemo.collections.asr.modules.AudioToMelSpectrogramPreprocessor
  normalize: per_feature
  window_size: 0.02
  sample_rate: 16000
  window_stride: 0.01
  window: hann
  features: 64
  n_fft: 512
  frame_splicing: 1
  dither: 1.0e-05
  stft_conv: false
```

You might ask, why are we using `_target_`? Well, it is generally rare for the preprocessor, encoder, decoder and perhaps a few other details to be changed often from the command line when experimenting. In order to stabilize these settings, we enforce that our preprocessor will always be of type `AudioToMelSpectrogramPreprocessor` for this model by setting its `_target_` attribute in the config. In order to provide its parameters in the class constructor, we simply add them after `_target_`.

---------
Note, we can still change all of the parameters of this `AudioToMelSpectrogramPreprocessor` class from the command line using hydra, so we don't lose any flexibility once we decide what type of preprocessing class we want !

This also gives us a flexible way to instantiate parts of the model from just the config object !

In [ ]:
new_preprocessor_config = copy.deepcopy(cfg.preprocessor)
new_preprocessor = citrinet.from_config_dict(new_preprocessor_config)
print(new_preprocessor)

So how do we actually update our model's internal preprocessor with something new? Well, since NeMo Model's are just pytorch Modules, we can just replace their attribute !

In [ ]:
citrinet.preprocessor = new_preprocessor

In [ ]:
citrinet.summarize()

--------
This might look like nothing changed - because we didn't actually modify the config for the preprocessor at all ! But as we showed above, we can easily modify the config for the preprocessor, instantiate it from config, and then just set it to the model.

-------
**NOTE**: Preprocessors don't generally have weights, so this was easy, but say we want to replace a part of the model which actually has trained parameters?

Well, the above approach will still work, just remember the fact that the new module you inserted into `citrinet.encoder` or `citrinet.decoder` actually won't have pretrained weights. You can easily rectify that by loading the state dict for the module *before* you set it to the Model though!

### Preserving the new config

So we went ahead and updated the preprocessor of the model. We however also need to perform a crucial step - **preserving the updated config**!

Why do we want to do this? NeMo has many ways of saving and restoring its models, which we will discuss a bit later. All of them depend on having an updated config that defines the model in its entirety, so if we modify anything, we should also update the corresponding part of the config to safely save and restore models.

In [ ]:
# Update the config copy
cfg.preprocessor = new_preprocessor_config
# Update the model config
citrinet.cfg = cfg

## Update a few special components of the Model
---------

While the above approach is good for most major components of the model, NeMo has special utilities for a few components.

They are -

 - `setup_training_data`
 - `setup_validation_data` and `setup_multi_validation_data`
 - `setup_test_data` and `setup_multi_test_data`
 - `setup_optimization`

These special utilities are meant to help you easily setup training, validation, testing once you restore a model from a checkpoint.

------
One of the major tasks of all conversational AI models is fine-tuning onto new datasets - new languages, new corpus of text, new voices etc. It is often insufficient to have just a pre-trained model. So these setup methods are provided to enable users to adapt models *after* they have been already trained or provided to you.




You might remember having seen a few warning messages the moment you tried to instantiate the pre-trained model. Those warnings are in fact reminders to call the appropriate setup methods for the task you want to perform.

Those warnings are simply displaying the old config that was used to train that model, and are a basic template that you can easily modify. You have the ability to modify the `train_ds`, `validation_ds` and `test_ds` sub-configs in their entirety in order to evaluate, fine-tune or train from scratch the model, or any further purpose as you require it.



Let's discuss how to add the scheduler to the model below (which initially had just an optimizer in its config)

In [ ]:
# Let's print out the current optimizer
print(OmegaConf.to_yaml(citrinet.cfg.optim))

In [ ]:
# Now let's update the config
citrinet.setup_optimization(cfg.optim)

-------
We see a warning -

```
Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !
```

We don't have a train dataset setup, nor do we have max_steps in the config. Most NeMo schedulers cannot be instantiated without computing how many train steps actually exist!

Here, we can temporarily allow the scheduler construction by explicitly passing a max_steps value to be 100

In [ ]:
OmegaConf.set_struct(cfg.optim.sched, False)

cfg.optim.sched.max_steps = 100

OmegaConf.set_struct(cfg.optim.sched, True)

In [ ]:
# Now let's update the config and try again
citrinet.setup_optimization(cfg.optim)

You might wonder why we didn't explicitly set `citrinet.cfg.optim = cfg.optim`.

This is because the `setup_optimization()` method does it for you! You can still update the config manually.

### Optimizer & Scheduler Config

Optimizers and schedulers are common components of models, and are essential to train the model from scratch.

They are grouped together under a unified `optim` namespace, as schedulers often operate on a given optimizer.



### Let's breakdown the general `optim` structure
```yaml
optim:
    name: novograd
    lr: 0.01

    # optimizer arguments
    betas: [0.8, 0.25]
    weight_decay: 0.001

    # scheduler setup
    sched:
      name: CosineAnnealing

      # Optional arguments
      max_steps: -1 # computed at runtime or explicitly set here
      monitor: val_loss
      reduce_on_plateau: false

      # scheduler config override
      warmup_steps: 1000
      warmup_ratio: null
      min_lr: 1e-9
```

Essential Optimizer components -

 - `name`: String name of the optimizer. Generally a lower case of the class name.
 - `lr`: Learning rate is a required argument to all optimizers.

Optional Optimizer components - after the above two arguments are provided, any additional arguments added under `optim` will be passed to the constructor of that optimizer as keyword arguments

 - `betas`: List of beta values to pass to the optimizer
 - `weight_decay`: Optional weight decay passed to the optimizer.

Optional Scheduler components - `sched` is an optional setup of the scheduler for the given optimizer.

If `sched` is provided, only one essential argument needs to be provided :

 - `name`: The name of the scheduler. Generally, it is the full class name.

Optional Scheduler components -

 - `max_steps`: Max steps as an override from the user. If one provides `trainer.max_steps` inside the trainer configuration, that value is used instead. If neither value is set, the scheduler will attempt to compute the `effective max_steps` using the size of the train data loader. If that too fails, then the scheduler will not be created at all.

 - `monitor`: Used if you are using an adaptive scheduler such as ReduceLROnPlateau. Otherwise ignored. Defaults to `loss` - indicating train loss as monitor.

 - `reduce_on_plateau`: Required to be set to true if using an adaptive scheduler.

Any additional arguments under `sched` will be supplied as keyword arguments to the constructor of the scheduler.




## Difference between the data loader setup methods
----------

You might notice, we have multiple setup methods for validation and test data sets. We also don't have an equivalent `setup_multi_train_data`.

In general, the `multi` methods refer to multiple data sets / data loaders.


### Where's `setup_multi_train_data`?
With the above in mind, let's tackle why we don't have `setup_multi_train_data`.

NeMo is concerned with multiple domains - `asr`, `nlp` and `tts`. The way datasets are setup and used in these domains is dramatically different. It is often unclear what it means to have multiple train datasets - do we concatenate them? Do we randomly sample (with same or different probability) from each of them?

Therefore we leave such support for multiple datasets up to the model itself. For example, in ASR, you can concatenate multiple train manifest files by using commas when providing the `manifest_filepath` value!

### What are multi methods?

In many cases, especially true for ASR and NLP, we may have multiple validation and test datasets. The most common example for this in ASR is `Librispeech`, which has `dev_clean`, `dev_other`, `test_clean`, `test_other`.

NeMo standardizes how to handle multiple data loaders for validation and testing, so that all of our collections have a similar look and feel, as well as ease development of our models. During evaluation, these datasets are treated independently and prepended with resolved names so that logs are separate!

The `multi` methods are therefore generalizations of the single validation and single test data setup methods, with some additional functionality. If you provide multiple datasets, you still have to write code for just one dataset and NeMo will automatically attach the appropriate names to your logs so you can differentiate between them!

Furthermore, they also automatically preserve the config the user passes to them when updating the validation or test data loaders.

**In general, it is preferred to call the `setup_multi_validation_data` and `setup_multi_test_data` methods, even if you are only using single datasets, simply for the automated management they provide.**

## Creating Model from constructor vs restoring a model
---------

You might notice, we discuss all of the above setup methods in the context of model after it is restored. However, NeMo scripts do not call them inside any of the example train scripts themselves.

This is because these methods are automatically called by the constructor when the Model is created for the first time, but these methods are skipped during restoration (either from a PyTorch Lightning checkpoint using `load_from_checkpoint`, or via `restore_from` method inside NeMo Models).

This is done as most datasets are stored on a user's local directory, and the path to these datasets is set in the config (either set by default, or set by Hydra overrides). On the other hand, the models are meant to be portable. On another user's system, the data might not be placed at exactly the same location, or even on the same drive as specified in the model's config!

Therefore we allow the constructor some brevity and automate such dataset setup, whereas restoration warns that data loaders were not set up and provides the user with ways to set up their own datasets.

------

Why are optimizers not restored automatically? Well, optimizers themselves don't face an issue, but as we saw before, schedulers depend on the number of train steps in order to calculate their schedule.

However, if you don't wish to modify the optimizer and scheduler, and prefer to leave them to their default values, that's perfectly alright. The `setup_optimization()` method is automatically called by PyTorch Lightning for you when you begin training your model!

## Saving and restoring models
----------

NeMo provides a few ways to save and restore models. If you utilize the Experiment Manager that is part of all NeMo train scripts, PyTorch Lightning will automatically save checkpoints for you in the experiment directory.

We can also use packaged files using the specialized `save_to` and `restore_from` methods.

### Saving and Restoring from PTL Checkpoints
----------

The PyTorch Lightning Trainer object will periodically save checkpoints when the experiment manager is being used during training.

PyTorch Lightning checkpoints can then be loaded and evaluated / fine-tuned just as always using the class method `load_from_checkpoint`.

For example, restore a Citrinet model from a checkpoint -

```python
citrinet = nemo_asr.models.EncDecCTCModelBPE.load_from_checkpoint(<path to checkpoint>)
```

### Saving and Restoring from .nemo files
----------

There are a few models which might require external dependencies to be packaged with them in order to restore them properly.

One such example is an ASR model with an external BPE tokenizer. It is preferred if the model includes all of the components required to restore it, but a binary file for a tokenizer cannot be serialized into a PyTorch Lightning checkpoint.

In such cases, we can use the `save_to` and `restore_from` method to package the entire model + its components (here, the tokenizer file(s)) into a tarfile. This can then be easily imported by the user and used to restore the model.

In [ ]:
# Save the model
citrinet.save_to('citrinet_512.nemo')

In [ ]:
!ls -d -- *.nemo

In [ ]:
# Restore the model
temp_cn = nemo_asr.models.EncDecCTCModelBPE.restore_from('citrinet_512.nemo')

In [ ]:
temp_cn.summarize()

In [ ]:
# Note that the preprocessor + optimizer config have been preserved after the changes we made !
print(OmegaConf.to_yaml(temp_cn.cfg))

Note, that .nemo file is a simple .tar.gz with checkpoint, configuration and, potentially, other artifacts such as tokenizer configs being used by the model

In [ ]:
!cp citrinet_512.nemo citrinet_512.tar.gz
!tar -xvf citrinet_512.tar.gz

### Extracting PyTorch checkpoints from NeMo tarfiles (Model level)
-----------

While the .nemo tarfile is an excellent way to have a portable model, sometimes it is necessary for researchers to have access to the basic PyTorch save format. NeMo aims to be entirely compatible with PyTorch, and therefore offers a simple method to extract just the PyTorch checkpoint from the .nemo tarfile.

In [ ]:
import torch

In [ ]:
state_dict = temp_cn.extract_state_dict_from('citrinet_512.nemo', save_dir='./pt_ckpt/')
!ls ./pt_ckpt/

As we can see below, there is now a single basic PyTorch checkpoint available inside the `pt_ckpt` directory, which we can use to load the weights of the entire model as below

In [ ]:
temp_cn.load_state_dict(torch.load('./pt_ckpt/model_weights.ckpt'))

### Extracting PyTorch checkpoints from NeMo tarfiles (Module level)
----------

While the above method is exceptional when extracting the checkpoint of the entire model, sometimes there may be a necessity to load and save the individual modules that comprise the Model.

The same extraction method offers a flag to extract the individual model level checkpoints into their individual files, so that users have access to per-module level checkpoints.

In [ ]:
state_dict = temp_cn.extract_state_dict_from('citrinet_512.nemo', save_dir='./pt_module_ckpt/', split_by_module=True)
!ls ./pt_module_ckpt/

Now, we can load and assign the weights of the individual modules of the above Citrinet Model !

In [ ]:
temp_cn.preprocessor.load_state_dict(torch.load('./pt_module_ckpt/preprocessor.ckpt'))
temp_cn.encoder.load_state_dict(torch.load('./pt_module_ckpt/encoder.ckpt'))
temp_cn.decoder.load_state_dict(torch.load('./pt_module_ckpt/decoder.ckpt'))

# NeMo with Hydra

[Hydra](https://hydra.cc/docs/intro/) is used throughout NeMo as a way to enable rapid prototyping using predefined config files. Hydra and OmegaConf offer great compatibility with each other, and below we show a few general helpful tips to improve productivity with Hydra when using NeMo.

## Hydra Help
--------

Since our scripts are written with hydra in mind, you might notice that using `python <script.py> --help` returns you a config rather than the usual help format from argparse.

Using `--help` you can see the default config attached to the script - every NeMo script has at least one default config file attached to it. This gives you a guide on how you can modify values for an experiment.

Hydra also has a special `--hydra-help` flag, which will offer you more help with respect to hydra itself as it is set up in the script.


## Changing config paths and files
---------

While all NeMo models come with at least 1 default config file, one might want to switch configs without changing code. This is easily achieved by the following commands :

- `--config-path`: Path to the directory which contains the config files
- `--config-name`: Name of the config file we wish to load.

Note that these two arguments need to be at the very beginning of your execution statement, before you provide any command line overrides to your config file.

## Overriding config from the command line
----------

Hydra allows users to provide command line overrides to any part of the config. There are three cases to consider -

 - Override existing value in config
 - Add new value in config
 - Remove old value in config

### Overriding existing values in config

Let's take the case where we want to change the optimizer from `novograd` to `adam`. Let's also change the beta values to default adam values.

Hydra overrides are based on the `.` syntax - each `.` representing a level in the config itself.

```sh
$ python <script>.py \
  --config-path="dir to config" \
  --config-name="name of config" \
  model.optim.name="adam" \
  model.optim.betas=[0.9,0.999]
```

It is to be noted, if lists are passed, there cannot be any spaces between items.

------

We can also support multi validation datasets with the above list syntax, but it depends on the model level support.

For ASR collection, the following syntax is widely supported in ASR, ASR-BPE and classification models. Let's take an example of a model being trained on LibriSpeech -

```sh
$ python <script>.py \
  --config-path="dir to config" \
  --config-name="name of config" \
  model.validation_ds.manifest_filepath=["path to dev clean","path to dev other"] \
  model.test_ds.manifest_filepath=["path to test clean","path to test other"]
```

### Add new values in config
----------

Hydra allows us to inject additional parameters inside the config using the `+` syntax.

Let's take an example of adding `amsgrad` fix for the `novograd` optimizer above.

```sh
$ python <script>.py \
  --config-path="dir to config" \
  --config-name="name of config" \
  +model.optim.amsgrad=true
```

### Remove old value in config
---------

Hydra allows us to remove parameters inside the config using the `~` syntax.

Let's take an example of removing `weight_decay` inside the Novograd optimizer

```sh
$ python <script>.py \
  --config-path="dir to config" \
  --config-name="name of config" \
  ~model.optim.weight_decay
```

## Setting a value to `None` from the command line

We may sometimes choose to disable a feature by setting the value to `None`.

We can accomplish this by using the keyword `null` inside the command line.

Let's take an example of disabling the validation data loader inside an ASR model's config -


```sh
$ python <script>.py \
  --config-path="dir to config" \
  --config-name="name of config" \
  model.test_ds.manifest_filepath=null
```

# NeMo Examples

NeMo supports various pre-built models for ASR, NLP and TTS tasks. One example we see in this notebook is the ASR model for Speech to Text - by using the Citrinet model.

The NeMo repository has a dedicated `examples` directory with scripts to train and evaluate models for various tasks - ranging from ASR speech to text, NLP question answering and TTS text to speech using models such as `FastPitch` and `HiFiGAN`.

NeMo constantly adds new models and new tasks to these examples, such that these examples serve as the basis to train and evaluate models from scratch with the provided config files.

NeMo Examples directory can be found here - https://github.com/NVIDIA/NeMo/tree/main/examples

## Structure of NeMo Examples
-------

The NeMo Examples directory is structured by domain, as well as sub-task. Similar to how we partition the collections supported by NeMo, the examples themselves are separated initially by domain, and then by sub-tasks of that domain.

All these example scripts are bound to at least one default config file. These config files contain all of the information of the model, as well as the PyTorch Lightning Trainer configuration and Experiment Manager configuration.

In general, once the model is trained and saved to a PyTorch Lightning checkpoint, or to a .nemo tarfile, it will no longer contain the training configuration - no configuration information for the Trainer or Experiment Manager.

**These config files have good defaults pre-set to run an experiment with NeMo, so it is advised to base your own training configuration on these configs.**


Let's take a deeper look at some of the examples inside each domain.



## ASR Examples
-------

NeMo supports multiple Speech Recognition models such as Jasper, QuartzNet, Citrinet, Conformer and more, all of which can be trained on various datasets. We also provide pretrained checkpoints for these models trained on standard datasets so that they can be used immediately. These scripts are made available in `speech_to_text_ctc.py`.

ASR examples also supports sub-tasks such as speech classification - MatchboxNet trained on the Google Speech Commands Dataset is available in `speech_to_label.py`. Voice Activity Detection is also supported with the same script, by simply changing the config file passed to the script!

NeMo also supports training Speech Recognition models with Byte Pair/Word Piece encoding of the corpus, via the `speech_to_text_ctc_bpe.py` example. Since these models are still under development, their configs fall under the `experimental/configs` directory.

Finally, in order to simply perform inference on some dataset using these models, prefer to use the `speech_to_text_eval.py` example, which provides a look at how to compute WER over a dataset provided by the user.

## NLP Examples
---------

NeMo supports a wide variety of tasks in NLP - from text classification and language modelling all the way to glue benchmarking!

All NLP models require text tokenization as data preprocessing steps. The list of tokenizers can be found in nemo.collections.common.tokenizers, and include WordPiece tokenizer, SentencePiece tokenizer or simple tokenizers like Word tokenizer.

A non-exhaustive list of tasks that NeMo currently supports in NLP is -

 - Language Modelling - Assigns a probability distribution over a sequence of words. Can be either generative e.g. vanilla left-right-transformer or BERT with a masked language model loss.
 - Text Classification - Classifies an entire text based on its content into predefined categories, e.g. news, finance, science etc. These models are BERT-based and can be used for applications such as sentiment analysis, relationship extraction
 - Token Classification - Classifies each input token separately. Models are based on BERT. Applications include named entity recognition, punctuation and capitalization, etc.
 - Intent Slot Classification - used for joint recognition of Intents and Slots (Entities) for building conversational assistants.
 - Question Answering - Currently only SQuAD is supported. This takes in a question and a passage as input and predicts a span in the passage, from which the answer is extracted.
 - Glue Benchmarks - A benchmark of nine sentence- or sentence-pair language understanding tasks


## TTS Examples
---------

NeMo supports Text To Speech (TTS, aka Speech Synthesis) via a two step inference procedure. First, a model is used to generate a mel spectrogram from text. Second, a model is used to generate audio from a mel spectrogram.

Supported Models:

Mel Spectrogram Generators:
*   Tacotron2
*   FastPitch
*   Talknet
*   And more...

Audio Generators (Vocoders):
*   WaveGlow
*   HiFiGAN
*   And more...

# NeMo Tutorials

Alongside the example scripts provided above, NeMo provides in depth tutorials for usage of these models for each of the above domains inside the `tutorials` directory found in the NeMo repository.

Tutorials are meant to be more in-depth explanation of the workflow in the discussed task - usually involving a small amount of data to train a small model on a task, along with some explanation of the task itself.

While the tutorials are a great example of the simplicity of NeMo, please note for the best performance when training on real datasets, we advice the use of the example scripts instead of the tutorial notebooks.

NeMo Tutorials directory can be found here - https://github.com/NVIDIA/NeMo/tree/main/tutorials